In [56]:
import random
import logging
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd

In [57]:
def requestPage(url: str):
        '''
        HTTP GET page with rotating user agents
        '''
        user_agent_list = [
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        ]
        next_user_agent = random.choice(user_agent_list)

        try:
            header = {'user-agent': next_user_agent}
            response = requests.get(url, headers=header)  # Get page with random user agent
        except requests.exceptions.ConnectionError:
            logging.error("ConnectionError: Likely too many simultaneous connections")
            logging.warning("Not all pages have been scraped")
            return None

        if response.status_code != 200:
            logging.error("RESPONSE {} ON >>> {}".format(response.status_code, url))
            logging.warning("Not all pages have been scraped")
            return None

        return response

def toSoup(response):
        '''
        Make sure lxml parser has been installed
        '''
        return BeautifulSoup(response.text, "html.parser")

In [58]:
response = requestPage("https://www.parkrun.org.uk/parkrunner/2813127/all/")
soup = toSoup(response)

In [59]:
tables = soup.find_all('table', {'id': 'results'})
results = tables[2]

In [63]:
df = pd.read_html(str(results))[0]
df.drop(columns=['PB?'])
df["Run Date"] = pd.to_datetime(df["Run Date"], format='%d/%m/%Y')
df["Time"].replace({r'\d\d:' : ''}, inplace=True, regex=True)
df.head()

,Event,Run Date,Run Number,Pos,Time,AgeGrade,PB?
0,Frimley Lodge,2022-06-04,571,38,42,63.69%,NaN
1,Southampton,2022-05-28,443,89,56,62.98%,NaN
2,Southampton,2022-05-21,442,123,27,61.46%,NaN
3,Southampton,2022-05-14,441,95,22,61.70%,NaN
4,Southampton,2022-05-07,440,115,00,59.92%,NaN


## Bingo?